In [1]:
!pip install pandas numpy scipy 

In [8]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output


# 1. DATA LOADING AND PREPARATION 

df = pd.read_csv(r"C:\Users\DELL\Desktop\data\ind30_m_vw_rets (1).csv", header=0, index_col=0, parse_dates=True)
size_df = pd.read_csv(r"C:\Users\DELL\Desktop\data\ind30_m_size.csv", header=0, index_col=0, parse_dates=True)

# Clean and align data
df.columns = df.columns.str.strip()
size_df.columns = size_df.columns.str.strip()
df = df / 100
df[df == -0.9999] = np.nan
df = df.dropna(how='all')
common_dates = df.index.intersection(size_df.index)
df = df.loc[common_dates]
size_df = size_df.loc[common_dates]
n_assets = len(df.columns)
print("Data loaded successfully.") #added this because i was struggling with importing the csv files into  jupyter labs so like a check


# 2. HELPER FUNCTIONS 
#(instead of just taking absolute views like hlthcare gives  15% ,i added factor exposure as well , the factors being low vol and momentumn 
#also to write the functions i have taken help from the codes provided in the course mentioned in the resources docs ( the second course)
def get_factor_views(returns_data, long_quantile=0.3, short_quantile=0.3, momentum_outperformance=0.04, low_vol_outperformance=0.02):
    num_assets = returns_data.shape[1]
    if len(returns_data) < 13:
        p_momentum = pd.Series(0.0, index=returns_data.columns)
    else:
        momentum = (1 + returns_data.iloc[-12:-1]).prod() - 1
        high_mom = momentum[momentum >= momentum.quantile(1 - long_quantile)]
        low_mom = momentum[momentum <= momentum.quantile(short_quantile)]
        p_momentum = pd.Series(0.0, index=returns_data.columns)
        if not high_mom.empty and not low_mom.empty:
            p_momentum.loc[high_mom.index] = 1 / len(high_mom)
            p_momentum.loc[low_mom.index] = -1 / len(low_mom)
    vols = returns_data.std() * np.sqrt(12)
    low_vol = vols[vols <= vols.quantile(short_quantile)]
    high_vol = vols[vols >= vols.quantile(1 - long_quantile)]
    p_low_vol = pd.Series(0.0, index=returns_data.columns)
    if not low_vol.empty and not high_vol.empty:
        p_low_vol.loc[low_vol.index] = 1 / len(low_vol)
        p_low_vol.loc[high_vol.index] = -1 / len(high_vol)
    P_factors = np.array([p_momentum.values, p_low_vol.values])
    Q_factors = np.array([[momentum_outperformance / 12], [low_vol_outperformance / 12]])
    return P_factors, Q_factors

def get_black_litterman_weights(returns_window, size_window, p_matrix, q_vector, risk_free_rate=0.02, delta=2.5, tau=0.05, max_weight=0.20):
    S = returns_window.cov()
    market_caps = size_window.iloc[-1]
    market_weights = market_caps / market_caps.sum()
    implied_returns = delta * S.dot(market_weights)
    omega = np.diag(np.diag(tau * p_matrix @ S @ p_matrix.T))
    if omega.ndim < 2: omega = np.array([[omega]])
    inv_tau_S = np.linalg.inv(tau * S)
    inv_omega = np.linalg.inv(omega)
    term1_inv = np.linalg.inv(inv_tau_S + p_matrix.T @ inv_omega @ p_matrix)
    term2 = inv_tau_S @ implied_returns + p_matrix.T @ inv_omega @ q_vector.flatten()
    posterior_returns = term1_inv @ term2
    n_assets = len(returns_window.columns)
    def neg_sharpe_ratio(weights, exp_returns, cov_matrix, rf_rate):
        p_ret = weights.T @ exp_returns
        p_vol = np.sqrt(weights.T @ cov_matrix @ weights)
        if p_vol == 0: return -np.inf
        return -(p_ret - rf_rate) / p_vol
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, max_weight) for x in range(n_assets))
    optimal_portfolio = minimize(neg_sharpe_ratio, market_weights.values,
                                 args=(posterior_returns, S, risk_free_rate/12),
                                 method='SLSQP', bounds=bounds, constraints=constraints)
    return optimal_portfolio.x, posterior_returns, implied_returns

def calculate_performance_metrics(returns_series, risk_free_rate=0.02):
    metrics = {}
    annualized_return = returns_series.mean() * 12
    annualized_volatility = returns_series.std() * np.sqrt(12)
    metrics['Annualized Return'] = annualized_return
    metrics['Annualized Volatility'] = annualized_volatility
    metrics['Sharpe Ratio'] = (annualized_return - risk_free_rate) / annualized_volatility
    cum_returns = (1 + returns_series).cumprod()
    peak = cum_returns.expanding(min_periods=1).max()
    drawdown = (cum_returns/peak) - 1
    metrics['Max Drawdown'] = drawdown.min()
    return pd.Series(metrics)

# 3. CORE BACKTESTING AND VISUALIsATION LOGI

def run_backtest_and_plot(health_view, mom_view, lowvol_view, delta, tau, max_weight):
    """
    This function runs the entire backtests and generate all outputs
    based on the parameters passed on from the ipywidgets.
    """
    print("Starting Backtest")
    
    # Backtest Parameters(we can change the look back window to 30 days or 90 days as well or even add a floatslider , but it would make a bit of mess in the final dashboard with so many floatsliders)
    lookback_window = 60
    rebalance_period = 12
    start_index = lookback_window

    # efine views based on widget inputs
    q_static = np.array([[health_view / 12]]) # Annual to monthly
    p_static = np.zeros((1, n_assets))
    p_static[0, df.columns.get_loc('Hlth')] = 1

    bl_portfolio_returns = []
    final_optimal_weights, final_posterior_returns, final_implied_returns = None, None, None

    for i in range(start_index, len(df), rebalance_period):
        if i + rebalance_period > len(df): break
        train_returns = df.iloc[i - lookback_window : i]
        train_size = size_df.iloc[i - lookback_window : i]
        test_returns = df.iloc[i : i + rebalance_period]

        try:
            p_factors, q_factors = get_factor_views(train_returns, 
                                                    momentum_outperformance=mom_view, 
                                                    low_vol_outperformance=lowvol_view)
            final_p = np.vstack([p_static, p_factors])
            final_q = np.vstack([q_static, q_factors])
            
            optimal_weights, posterior_returns, implied_returns = get_black_litterman_weights(
                returns_window=train_returns, size_window=train_size, p_matrix=final_p, q_vector=final_q,
                delta=delta, tau=tau, max_weight=max_weight
            )
            final_optimal_weights, final_posterior_returns, final_implied_returns = optimal_weights, posterior_returns, implied_returns
        except np.linalg.LinAlgError:
            print(f"Singular matrix encountered for period starting {df.index[i].date()}. Using equal weights.")
            optimal_weights = np.repeat(1/n_assets, n_assets)
            
        period_returns = test_returns.dot(optimal_weights)
        bl_portfolio_returns.append(period_returns)

    if not bl_portfolio_returns:
        print("Backtest did not complete. Cannot generate results.")
        return

    bl_returns_series = pd.concat(bl_portfolio_returns)
    bl_returns_series.name = "Black-Litterman"
    print("\nBacktest Complete")

    #Performance Analysis 
    ew_weights = np.repeat(1/n_assets, n_assets)
    ew_returns_series = df.loc[bl_returns_series.index].dot(ew_weights)
    ew_returns_series.name = "Equal-Weighted"
    backtest_results = pd.concat([bl_returns_series, ew_returns_series], axis=1)
    performance_summary = backtest_results.apply(calculate_performance_metrics)
    print("\n--- Performance Summary ---")
    print(performance_summary)

    #Visualisation/graphs and plots to display the meterics
    cumulative_returns = (1 + backtest_results).cumprod()
    plt.figure(figsize=(14, 7))
    cumulative_returns.plot(ax=plt.gca())
    plt.title("Backtest: BL with Factors vs. Equal-Weighted Portfolio", fontsize=16)
    plt.ylabel("Cumulative Growth of $1")
    plt.grid(True, linestyle='--', alpha=0.6); plt.legend(); plt.show()

    if final_optimal_weights is not None:
        plt.figure(figsize=(12, 6))
        pd.Series(final_optimal_weights, index=df.columns).sort_values().plot(kind='barh')
        plt.title('Optimal Portfolio Weights (Last Rebalance Period)'); plt.xlabel('Weight'); plt.show()

    if final_implied_returns is not None and final_posterior_returns is not None:
        returns_comparison = pd.DataFrame({'Implied': final_implied_returns, 'Posterior': final_posterior_returns})
        returns_comparison.sort_values('Posterior').plot(kind='bar', figsize=(15, 7))
        plt.title('Implied vs. Posterior Expected Returns (Last Rebalance Period)'); plt.ylabel('Expected Return'); plt.show()

# 4. SETTING UP IPYWIODGETS DASHBOARD


# Defining  widgets we want so see on dashboard
style = {'description_width': 'initial'}
# Parameters
delta_slider = widgets.FloatSlider(value=2.5, min=1.0, max=5.0, step=0.1, description='Risk Aversion (delta)', style=style)
tau_slider = widgets.FloatSlider(value=0.05, min=0.01, max=0.5, step=0.01, description='Confidence in Views (tau)', style=style)
max_weight_slider = widgets.FloatSlider(value=20, min=5, max=100, step=1, description='Max Asset Weight (%)', style=style)
health_view_slider = widgets.FloatSlider(value=10, min=-5, max=25, step=1, description='Hlth Annual Return View (%)', style=style)
mom_view_slider = widgets.FloatSlider(value=4, min=0, max=10, step=0.5, description='Momentum Outperformance View (%)', style=style)
lowvol_view_slider = widgets.FloatSlider(value=2, min=0, max=10, step=0.5, description='Low Vol Outperformance View (%)', style=style)

# Button to trigger/activate the backtest
run_button = widgets.Button(description="Run Backtest", button_style='success')

# Output area to display results
output_area = widgets.Output()

# Define a function to be called on button click
def on_button_click(b):
    # Clear the previous output
    output_area.clear_output(wait=True)
    
    # Running  backtest inside the output area
    with output_area:
        # Get current values from sliders and convert percentages to decimals
        run_backtest_and_plot(
            health_view=health_view_slider.value / 100,
            mom_view=mom_view_slider.value / 100,
            lowvol_view=lowvol_view_slider.value / 100,
            delta=delta_slider.value,
            tau=tau_slider.value,
            max_weight=max_weight_slider.value / 100
        )

# connect the function to the button
run_button.on_click(on_button_click)

# Arrange widgets into a dashboard layout
param_box1 = widgets.VBox([delta_slider, tau_slider, max_weight_slider])
param_box2 = widgets.VBox([health_view_slider, mom_view_slider, lowvol_view_slider])
controls = widgets.HBox([param_box1, param_box2])

# Display the final dashboard
dashboard = widgets.VBox([
    widgets.HTML("<h2>Black-Litterman Interactive Dashboard</h2>"),
    widgets.HTML("<p>Adjust the model and view parameters below, then click the button to run the simulation.</p>"),
    controls,
    run_button,
    output_area
])

# Display the dashboard in the jupyter notebook(finally)
display(dashboard)


Data loaded successfully.


C:\Users\DELL\AppData\Local\Temp\ipykernel_13320\884872836.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(r"C:\Users\DELL\Desktop\data\ind30_m_vw_rets (1).csv", header=0, index_col=0, parse_dates=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_13320\884872836.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  size_df = pd.read_csv(r"C:\Users\DELL\Desktop\data\ind30_m_size.csv", header=0, index_col=0, parse_dates=True)
